In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def select_subjects(df, sites=None, sex=None, age_range=None, group=None):
    '''
    pass
    '''
    
    dictionary = d = {'m': 1, 'f': 2, 'ASD': 1, 'HC': 2} #dict for transforming the inputs to values found in the df
    #Fill the variables in such a way that it selects all subjects
    if sites == None: 
        sites = list(df.SITE_ID.unique())
    if sex == None:
        sex = ['m','f']
    if age_range == None:
        age_range = (0,150)
    if group == None:
        group = ['HC','ASD']
    
    #Actual code
    if isinstance(group, list) == False:
        group = group.split()
    sex = list(map(dictionary.get,sex))
    group = list(map(dictionary.get,group))
    
    return df[(df['SITE_ID'].isin(sites)) & 
              (df['SEX'].isin(sex)) & 
              (df['AGE_AT_SCAN'].between(age_range[0], age_range[1])) & 
              (df['DX_GROUP'].isin(group)) 
             ]

In [3]:
Dosenbach_df = pd.read_csv('BendDosenbach.csv',index_col=0)
AAL_df = pd.read_csv('BendAAL.csv',index_col=0)

In [39]:
data = select_subjects(Dosenbach_df,sex='m',sites=['YALE','PITT','NYU','TRINITY','STANFORD','UCLA1','SDSU','UM1'])
X = data.drop(['DX_GROUP','SEX','AGE_AT_SCAN','FILE_ID','SITE_ID','subject'],axis=1)
y = data['DX_GROUP']

In [5]:
from sklearn.model_selection import KFold
cv = KFold(n_splits=10,random_state=101)

In [43]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler,Normalizer
from sklearn.svm import SVC,LinearSVC
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

scaler = StandardScaler()
X = scaler.fit_transform(X)
svc_rbf = make_pipeline(LinearSVC(random_state = 101,dual=False,C=1,penalty='l1'))

sfs = SFS(svc_rbf, 
           k_features=(1,15), 
           forward=True, 
           floating=True, 
           verbose=2,
           scoring='accuracy',
           cv=cv,n_jobs=-1)

sfs = sfs.fit(X, y.values)

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 567 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 966 out of 966 | elapsed:    5.6s finished

[2018-08-22 17:08:06] Features: 1/15 -- score: 0.545448717948718[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 567 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done 965 out of 965 | elapsed:    5.4s finished
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.6s finished

[2018-08-22 17:08:13] Features: 2/15 -- score: 0.5931410256410257[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 964 out of 964 | elapsed:    5.4s finished
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    1.0s finished

[2018-08-22 17:08:20] Features: 3/15 -- score: 0.6084615384615384[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done 567 tasks      | elapsed:    4.3s
[Parallel(n_jo

In [36]:
from sklearn.model_selection import cross_val_score
cross_val_score(estimator=svc_rbf,X=X,y=y,cv=cv).mean()

0.49705128205128196

In [42]:
X

array([[ 0.0141776 ,  1.61319856,  1.56357253, ..., -1.22218066,
        -1.44835927, -0.42855828],
       [ 2.83899962,  1.325862  ,  1.48901451, ..., -0.59883618,
        -2.05777411, -1.18370647],
       [ 0.77798536,  1.11259987,  1.14503087, ...,  0.40084586,
         0.45542859,  0.30964144],
       ...,
       [-0.72690102, -0.60625486, -0.61724982, ...,  0.59541766,
         0.47954698,  1.1514903 ],
       [ 0.29381785, -0.36890201, -0.49016227, ...,  0.9849999 ,
         0.52113638,  0.73643496],
       [-0.82000164, -0.61080389, -0.40204824, ..., -0.35535005,
         0.65209648, -1.48878776]])